In [61]:
from datetime import datetime
from pathlib import Path
import pandas as pd

## Which cells/tissues were sequenced together vs separately?

does this contribute to differences in SLAV model performance?

In [85]:
# read in excel files from seq submissions and extract cell ids and sequencing dates
data = []
for f in Path("../resources/seq_submissions").rglob("*xls*"):
    # extract date from filename
    date = f.stem.split("_")[0]
    if len(date) == 8:
        date = datetime.strptime(date, "%m%d%Y")
    elif len(date) == 6:
        date = datetime.strptime(date, "%y%m%d")
    else:
        raise ValueError(f"Unknown date format: {date}")

    # read in excel file
    df = pd.read_excel(f, sheet_name="Pooled_libraries", skiprows=18, usecols=[1])
    df = df.rename(columns={df.columns[0]: "cell_id"})
    df["batch"] = date
    df = df.loc[~df.cell_id.isna(), :]
    data.append(df)

print(f"Stored {len(data)} files")
data = pd.concat(data).drop_duplicates().reset_index(drop=True)
data["duplicated"] = data.duplicated(subset=["cell_id"], keep=False)

/iblm/netapp/data2/mcuoco/conda/envs/shell/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/iblm/netapp/data2/mcuoco/conda/envs/shell/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/iblm/netapp/data2/mcuoco/conda/envs/shell/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/iblm/netapp/data2/mcuoco/conda/envs/shell/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Stored 20 files


In [87]:
data[data["duplicated"]].sort_values("cell_id").to_csv(
    "duplicated_cells.csv", index=False
)

In [89]:
data[data["duplicated"]].batch.value_counts()

batch
2019-01-04    479
2019-01-07    424
2019-02-05    279
2019-01-11    236
2019-05-28     44
2019-11-04     37
2020-03-02     37
Name: count, dtype: int64